##### To do list:
- ~~download/combine xml files~~
- ~~concatenate into DF~~
- ~~create a contest_list~~
- ~~subset contests of interest~~
- ~~add a fips column~~
- ~~create unique precinct id using COUNTYFIPS + precinct~~
- ~~standardize contest names~~
- ~~change column names to vest naming conventions~~
        NOTE: supreme court is a S election, lieutenant governor misspelled as 'liutenant' somewhere, 'Lagrua' misspelled as 'lagura' somewhere
- add col for incumbent (yes/no)
- pivot data
- create list of ordered candidates
- order df by contest size, votes won
- create readme
- statewide total checks
- countywide total checks
- export data to csv
- move helper functions to py module
- clean up code
- push to gh
- send to Spencer

## Georgia 2022 Primary Runoff Election Returns

### Sources
- [State Office of Elections Certified Report Summaries]
- [State Office of Elections Certified Precinct Detail Report- XML format]

In [4]:
import geopandas as gp
import pandas as pd
import os
import xml.etree.ElementTree as et
import numpy as np
import GA22_primaryrunoff_helper as hlp

## Load Raw Data, Convert to DataFrame

In [10]:
#Peter's XML parser
loaded_counties = os.listdir("./raw-from-source/counties/")
z=[]
for locale in loaded_counties:
    if locale.endswith('.xml'):
        file_string = "./raw-from-source/counties/"+locale
        xtree = et.parse(file_string)
        xroot = xtree.getroot()
        store_list = []
        county_area = xroot.findall(".//Region")
        for i in county_area:
            county = i.text
        contests = xroot.findall(".//Contest")
        for i in contests:
            contest = i.attrib.get('text')
            lower = i.findall("./Choice")
            for j in lower:
                choice = j.attrib.get('text')
                lower_2 = j.findall("./VoteType")
                for k in lower_2:
                    voting_method = k.attrib.get('name')
                    lower_3 = k.findall("./Precinct")
                    for l in lower_3:
                        precinct_name = l.attrib.get('name')
                        num_votes = l.attrib.get('votes')
                        z.append([county,contest,choice,voting_method,precinct_name,num_votes])
dfcols = ['county','contest','choice','voting_method','precinct','num_votes']
df_primary_runoff = pd.DataFrame(z,columns=dfcols)
df_primary_runoff["election"] = "primary runoff"
df_primary_runoff["precinct"] = df_primary_runoff["precinct"].str.strip()

Concatenate into df, check for 159 counties

In [11]:
ga_22_primary_runoff = pd.concat([df_primary_runoff])
#check
print(len(ga_22_primary_runoff["county"].unique()))

159


In [12]:
ga_22_primary_runoff.shape

(105184, 7)

In [13]:
ga_22_primary_runoff.head()

,county,contest,choice,voting_method,precinct,num_votes,election
0,Appling,Lieutenant Governor - Dem,Charlie Bailey,Absentee by Mail Votes,1B,1,primary runoff
1,Appling,Lieutenant Governor - Dem,Charlie Bailey,Absentee by Mail Votes,1C,0,primary runoff
2,Appling,Lieutenant Governor - Dem,Charlie Bailey,Absentee by Mail Votes,2,7,primary runoff
3,Appling,Lieutenant Governor - Dem,Charlie Bailey,Absentee by Mail Votes,3A1,2,primary runoff
4,Appling,Lieutenant Governor - Dem,Charlie Bailey,Absentee by Mail Votes,3C,5,primary runoff
